# MOFA Pipeline

- Here is a useful tutorial to get started running MOFApy: https://github.com/bioFAM/mofapy2/blob/master/mofapy2/notebooks/getting_started_python.ipynb

- Here is a useful tutorial to evaluate the MOFApy results using mofax: https://github.com/bioFAM/mofax/blob/master/notebooks/getting_started_pbmc10k.ipynb



In [ ]:
from mofapy2.run.entry_point import entry_point
import pandas as pd
import numpy as np
import os
import scanpy as sc
import mofax as mfx

from umap import UMAP
from sklearn.decomposition import PCA
from scipy.stats import ranksums
from scipy.stats import zscore
import matplotlib.pyplot as plt
import seaborn as sns
import decoupler as dc
import plotnine as pn

from composition_stats import closure, ilr

# initialise the entry point
ent = entry_point()

In [ ]:
output = "cellbender"
deconv_model = "all"
dc_model = "ulm"
ct_metric = "abunds"
image_features = "None"
n_factors = 10
recompute = True

In [ ]:
#NOTE: hallmarks are always used
obsm_to_use = []
#TODO: rerun process and remove this line
#obsm_to_use.append(f"hallmark_{dc_model}_estimates")
if dc_model == "ulm":
    obsm_to_use.append("hallmark_ulm_estimates") # see here "estimates"
elif dc_model == "wmean":
    obsm_to_use.append("hallmark_wmean_estimate") # see here "estimate"

if image_features != "None":
    obsm_to_use.append(image_features)

obsm_to_use.append(f"{ct_metric}_{deconv_model}")

print(f"Using the following views: {obsm_to_use}")

In [ ]:
# set the paths
current_path = globals()["_dh"][0]
out_dir = current_path / ".." / ".." / "data" / "prc" / "vis" / "mofa_tests" / output / f"{deconv_model}__{ct_metric}__{dc_model}__{image_features}__{str(n_factors)}"
plot_dir = out_dir / "plots"
sc.settings.figdir = str(plot_dir)
out_dir.mkdir(parents=True, exist_ok=True)
plot_dir.mkdir(parents=True, exist_ok=True) 
visium_path = current_path / ".." / ".." / "data" / "prc" / "vis" / "processed" / output
visium_samples = [f.split(".")[0] for f in os.listdir(visium_path) if not f.startswith(".")]

In [ ]:
sample_meta = pd.read_excel(current_path / ".." / ".." / "data" / "Metadata_all.xlsx", sheet_name="Visium")
sample_meta

In [ ]:
vis_dict = {smp: sc.read_h5ad(visium_path / f"{smp}.h5ad") for smp in visium_samples}

In [ ]:
first_sample = visium_samples[0]
print(first_sample)
vis_dict[first_sample]

In [ ]:
vis_dict[first_sample].obsm_keys()

In [ ]:
# make scatterplot
plt.scatter(vis_dict["MS549H"].obsm["abunds_all"].sum(axis=0), vis_dict["MS497I"].obsm["abunds_all"].sum(axis=0))
plt.show()

In [ ]:
obsm_to_use = ["abunds_all", "hallmark_ulm_estimates"]
#obsm_to_use = ["abunds_all", "hallmark_estimates", "summary"]
#obsm_to_use = ["props_all_ilr", "hallmark_estimates"]
assert np.all(np.isin(obsm_to_use, vis_dict[first_sample].obsm_keys()))

In [ ]:
vis_dict[first_sample].obsm["abunds_all"].shape[1], vis_dict[first_sample].obsm["props_ilr_all"].shape[1]

In [ ]:
# TODO: temporary fix to put the ilr transformed data into a dataframe
# NOTE: It is important that the column names do not resemble integers otherwise there are hd5 errors when saving the MOFA model
for values in vis_dict.values():
    values.obsm["props_ilr_all"] = pd.DataFrame(values.obsm["props_ilr_all"], index=values.obs.index, 
                                                columns=["irl_" + str(s) for s in list(range(values.obsm["props_ilr_all"].shape[1]))])
    values.obsm["props_ilr_condition"] = pd.DataFrame(values.obsm["props_ilr_condition"], index=values.obs.index, 
                                                      columns=["irl_" + str(s) for s in list(range(values.obsm["props_ilr_condition"].shape[1]))])
    #values.obsm["props_ilr_lesion_type"] = pd.DataFrame(values.obsm["props_ilr_lesion_type"], index=values.obs.index, 
    #                                                    columns=["irl_" + str(s) for s in list(range(values.obsm["props_ilr_lesion_type"].shape[1]))])

In [ ]:
print(visium_samples[0])
print(f"Adata object for first visium sample:\n{vis_dict[visium_samples[0]]}")
print(f"Available obsm layers:\n{vis_dict[visium_samples[0]].obsm_keys()}")
obsm_features = {obsm_key: vis_dict[visium_samples[0]].obsm[obsm_key].columns.to_list() for obsm_key in obsm_to_use}

In [ ]:
# get cell metadata
meta_list = []
for sample in visium_samples:
    df = vis_dict[sample].obs.copy()
    df.index = [sample + "_" + s for s in df.index]
    df["sample_id"] = sample
    df["condition"] = sample_meta.loc[sample_meta.sample_id == sample, "Condition"].values[0]
    df["lesion_type"] = sample_meta.loc[sample_meta.sample_id == sample, "lesion_type"].values[0]
    meta_list.append(df)
meta_df = pd.concat(meta_list, axis=0)
meta_df

In [ ]:
# create mofa dataframe
df_list = []
for obsm_key in obsm_to_use:
    for sample in visium_samples:
        df = vis_dict[sample].obsm[obsm_key].copy()
        df.index = [sample + "_" + s for s in df.index] # unique barcodes are required!
        df = df.reset_index().melt(id_vars="index", var_name="feature", value_name="value")
        df = df.rename(columns={"index": "sample"})
        df["group"] = sample
        df["view"] = obsm_key
        df = df[["sample", "group", "feature", "value", "view"]]
        df_list.append(df)
data_dt = pd.concat(df_list)
print("Mofa dataframe:\n")
print(data_dt.head())
print(data_dt.tail())

In [ ]:
# scale each view to unit variance
ent.set_data_options(
    scale_views = True
)

# set the likelihoods for all views
ent.set_data_df(data_dt, likelihoods = ["gaussian" for _ in range(len(obsm_to_use))])

# set the model options
ent.set_model_options(
    factors = 10,
    spikeslab_weights = True, 
    ard_weights = True
)

# set the training options
ent.set_train_options(
    convergence_mode = "fast", 
    dropR2 = 0.001, 
    gpu_mode = True, 
    seed = 1
)

In [ ]:
ent.build()

In [ ]:
ent.run()

In [ ]:
ent.data_opts.keys()

In [ ]:
ent.save(outfile=str(out_dir / "mofa_model.hdf5"))

In [ ]:
model = mfx.mofa_model(out_dir / "mofa_model.hdf5")
model

In [ ]:
print(f"""\
Cells: {model.shape[0]}
Features: {model.shape[1]}
Groups of cells: {', '.join(model.groups)}
Views: {', '.join(model.views)}
""")

In [ ]:
# create adata object based on MOFA results
adata = sc.AnnData(X=model.get_factors())
adata.obs = model.get_cells()
adata.obs.set_index("cell", inplace=True)
sc.pp.neighbors(adata, n_neighbors=15, use_rep="X")
resolutions = [0.2, 0.3, 0.4, 0.5, 0.6]
for res in resolutions:
    sc.tl.leiden(adata, resolution=res, key_added=f"leiden_{res}")
adata.obs.rename(columns={"group": "sample_id"}, inplace=True)
adata.obs = adata.obs.join(sample_meta.set_index("sample_id"), on="sample_id")
sc.tl.umap(adata)

for obsm_key in obsm_to_use:
    df_list = []
    for sample in visium_samples:
        df = vis_dict[sample].obsm[obsm_key].copy()
        df.index = [sample + "_" + s for s in df.index] # unique barcodes are required!
        df_list.append(df)
    df = pd.concat(df_list)
    adata.obsm[obsm_key] = df.loc[adata.obs_names]
adata

In [ ]:
adata.write(out_dir / "adata.h5ad")

In [ ]:
sc.pl.umap(adata, color=["sample_id", "Condition", "leiden_0.5"], ncols=1, save=".pdf", show=True)

In [ ]:
# count the cluster fractions per sample, pca, and visualize
for res in resolutions:
    df = adata.obs.copy()
    df = df.groupby(["sample_id", f"leiden_{res}"]).size().unstack()
    df += 1 # adding pseudocount otherwise ilr breaks (returns NA)
    df = df.div(df.sum(axis=1), axis=0)

    # remove outlier sample MS94
    df = df.loc[df.index != "MS94"]

    # apply ilr (first check and ensure closure)
    df.loc[:, :] = closure(df.values)
    assert np.all(np.isclose(df.values.sum(axis=1), 1))
    df_ilr = ilr(df.values)

    pca = PCA(n_components=df_ilr.shape[1])
    pca.fit(df_ilr)
    pca_df = pd.DataFrame(pca.transform(df_ilr)[:,[0,1,2,3]], index=df.index, columns=["PC1", "PC2", "PC3", "PC4"])
    pca_df["sample_id"] = pca_df.index.get_level_values(0)
    pca_df.set_index("sample_id", inplace=True)
    pca_df = pca_df.join(sample_meta.set_index("sample_id"),  on="sample_id", how="left", lsuffix="_pca", rsuffix="_meta")
    pca_df = pca_df.reset_index()
    pca_df.Batch = [str(b) for b in pca_df.Batch]

    fig, ax = plt.subplots(3, 2, figsize=(14, 12))
    ax = ax.flatten()
    for axis, label in zip(ax, ["Condition", "lesion_type", "Batch", "Sex", "Age", "lesion_type"]):
        sns.scatterplot(data=pca_df, x="PC1", y="PC2", hue=label, ax=axis)
        axis.set_title(f"PCA on Cluster Proportions, colored by {label}")
        axis.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
        axis.set_xlabel(f"PC1: {np.round(pca.explained_variance_ratio_[0], 2)}")
        axis.set_ylabel(f"PC2: {np.round(pca.explained_variance_ratio_[1], 2)}")
        fig.tight_layout()
        axis.set_aspect('equal', 'box')
    # plot sample_id labels
    for string, x, y in zip(pca_df.sample_id, pca_df.PC1, pca_df.PC2):
        ax[5].text(x, y, string)
    fig.savefig(plot_dir / f"cluster_fractions_pca_{res}.pdf")

In [ ]:
# count the cluster fractions per sample, divide by the rowsums
for res in resolutions:
    df = adata.obs.groupby(["sample_id", f"leiden_{res}"]).size().unstack()
    df = df.div(df.sum(axis=1), axis=0)
    fig, ax = plt.subplots(figsize=(12, 8))
    sns.heatmap(df, cmap="viridis", annot=True, fmt=".2f", ax=ax)
    ax.set_xlabel("Cluster")
    ax.set_ylabel("Sample ID")
    fig.tight_layout()
    fig.suptitle(f"Cluster fractions per sample (resolution {res})")
    fig.savefig(plot_dir / f"cluster_fractions_per_sample_heatmap_res_{res}.pdf")

In [ ]:
# count the cluster fractions per lesion type, divide by the rowsums
for res in resolutions:
    df = adata.obs.groupby(["lesion_type", f"leiden_{res}"]).size().unstack()
    df = df.div(df.sum(axis=1), axis=0)
    fig, ax = plt.subplots(figsize=(12, 4))
    sns.heatmap(df, cmap="viridis", annot=True, fmt=".2f", ax=ax)
    ax.set_xlabel("Cluster")
    ax.set_ylabel("Lesion Type")
    fig.tight_layout()
    fig.suptitle(f"Cluster fractions per lesion type (resolution {res})")
    fig.savefig(plot_dir / f"cluster_fractions_per_lesion_type_heatmap_res_{res}.pdf")

In [ ]:
# count the cluster fractions per condition, divide by the rowsums
for res in resolutions:
    df = adata.obs.groupby(["Condition", f"leiden_{res}"]).size().unstack()
    df = df.div(df.sum(axis=1), axis=0)
    fig, ax = plt.subplots(figsize=(12, 2))
    sns.heatmap(df, cmap="viridis", annot=True, fmt=".2f", ax=ax)
    ax.set_xlabel("Cluster")
    ax.set_ylabel("Condition")
    fig.tight_layout()
    fig.suptitle(f"Cluster fractions per condition (resolution {res})")
    fig.savefig(plot_dir / f"cluster_fractions_per_condition_heatmap_res_{res}.pdf")

In [ ]:
# TODO: Implement what we have been taling about in the meeting with Pau

In [ ]:
#obsm_key = obsm_to_use[0]
#res = resolutions[0]
obsm_key = "abunds_all"
res = 0.5
obsm_key, res

In [ ]:
"abunds" in obsm_key

In [ ]:
df = adata.obsm[obsm_key].copy()
meta_keys = ["sample_id", "Condition", "lesion_type"] + ["leiden_" + str(res) for res in resolutions]
df = df.join(adata.obs[meta_keys], on="cell")
df.reset_index(inplace=True)
df = df.melt(id_vars=["cell"]+meta_keys, var_name="feature", value_name="value")
assert df.value.isna().sum() == 0
if "abunds" in obsm_key:
    # use proportions instead of abundances for cell types
    df["value"] = df.groupby(["cell"])["value"].transform(lambda x: x / x.sum())
df = df.groupby(["sample_id", f"leiden_{res}", "feature"]).mean().reset_index()
df.value.fillna(0, inplace=True) # replace NA in value column with 0
df

In [ ]:
leidens = df[f"leiden_{res}"].unique()
leidens

In [ ]:
features = df.feature.unique()
features

In [ ]:
pvals = []
for leiden in leidens:
        row = []
        msk = df[f"leiden_{res}"] == leiden
        for f in features:
            w, p = ranksums(df.value[df.feature==f][msk], df.value[df.feature==f][~msk], alternative='two-sided')
            row.append(p)
        row = dc.p_adjust_fdr(row)
        pvals.append(row)
pvals = pd.DataFrame(pvals, columns=features, index=leidens)
pvals

In [ ]:
pvals.loc[:, :] = np.where(pvals.values < 0.05, '*', '')

In [ ]:
# groupby leiden and get the mean per featue
df_plot = df.groupby([f"leiden_{res}", "feature"]).mean().reset_index()
# pivot the table
df_plot = df_plot.pivot(index=f"leiden_{res}", columns="feature", values="value")
df_plot.loc[:, :] = zscore(df_plot.values, axis=0, ddof=1)
df_plot

In [ ]:
# Define color map
cmap = plt.get_cmap('coolwarm').copy()
cmap.set_bad(color='gray')

# Plot
fig, ax = plt.subplots(1, 1, figsize=(4, 4), facecolor='white', dpi=125)
htm = sns.heatmap(df_plot.T, cmap=cmap, square=True, center=0, vmax=1, vmin=-1, ax=ax, cbar_kws={"shrink": .4, "aspect": 5},
                  annot=pvals.T.values.astype('U'), fmt='', annot_kws={'fontweight': 'black', 'color': 'black'})
i = 0
for _, spine in htm.spines.items():
    if i % 2 == 0:
        spine.set_visible(True)
    i += 1

In [ ]:
pvals

In [ ]:
pvals

In [ ]:
pvals

In [ ]:
# compute average features per cluster
for obsm_key in obsm_to_use:
    df = adata.obsm[obsm_key].copy()
    meta_keys = ["sample_id", "Condition", "lesion_type"] + ["leiden_" + str(res) for res in resolutions]
    df = df.join(adata.obs[meta_keys], on="cell")
    df.reset_index(inplace=True)
    df = df.melt(id_vars=["cell"]+meta_keys, var_name="feature", value_name="value")
    for res in resolutions:
        # TODO: add pvalues here to the heatmaps
        # TODO: save the test statistic dataframe
        # TODO: change the heatmap color to the one from the preprint
        ### start from https://github.com/schae211/VisiumMS/blob/4059a4c8e2af9c39af787ebee1439fc854d311d6/scripts/figures/fig2/heatmap_progeny.py#L30 ###
        #leidens = df[f"leiden_{res}"].unique()
        #pathways = acts.var_names
        #pvals = []
        #for leiden in leidens:
        #    row = []
        #    msk = acts.obs['leiden'] == leiden
        #    for pathway in pathways:
        #        w, p = ranksums(acts[:, pathway][msk].X, acts[:, pathway][~msk].X, alternative='greater')
        #        row.append(p[0])
        #    row = dc.p_adjust_fdr(row)
        #    pvals.append(row)
        #pvals = pd.DataFrame(pvals, columns=pathways, index=leidens)
        ### end ###
        df_tmp = df.groupby(["feature", f"leiden_{res}"]).mean().reset_index()
        df_tmp["min_max_value"] = df_tmp.groupby(["feature"]).value.apply(lambda x: (x - x.min()) / (x.max() - x.min()))
        fig, ax = plt.subplots(1, 2, figsize=(24, 12))
        sns.heatmap(data=df_tmp.pivot(index="feature", columns=f"leiden_{res}", values="value"), 
                    cmap="viridis", ax=ax[0])
        sns.heatmap(data=df_tmp.pivot(index="feature", columns=f"leiden_{res}", values="min_max_value"), 
                cmap="viridis", ax=ax[1])
        ax[0].set_title("Raw values")
        ax[1].set_title("Min-max scaled values per feature")
        fig.savefig(plot_dir / f"feature_heatmap_{obsm_key}_res_{res}.pdf")


In [ ]:
# compute feature loadings per factor
for obsm_key in obsm_to_use:
    features = obsm_features[obsm_key]
    fig, ax = plt.subplots(figsize=(6, 10))
    sns.heatmap(model.get_weights(df=True).loc[features, :], ax=ax, cmap="coolwarm", center=0)
    fig.tight_layout()
    fig.savefig(plot_dir / f"feature_loadings_{obsm_key}.pdf")

In [ ]:
# check the fraction of variance explained per group per factor per view
fig = mfx.plot.plot_r2(model, y="Group", x="Factor")
fig.tight_layout()
fig.savefig(plot_dir / "r2_per_group_per_factor_per_view.pdf")

In [ ]:
res = 0.5
df = adata.obs.copy()
df = df.groupby(["sample_id", f"leiden_{res}"]).size().unstack()
df += 1 # adding pseudocount otherwise ilr breaks
df = df.div(df.sum(axis=1), axis=0)

# remove outlier sample MS94
df = df.loc[df.index != "MS94"]

# check closure
df.loc[:, :] = closure(df.values)
assert np.all(np.isclose(df.values.sum(axis=1), 1))
#plt.imshow(df.values)
#plt.colorbar()
#plt.show()

# apply ilr
df_ilr = ilr(df.values)

#plt.imshow(df_ilr)
#plt.colorbar()
#plt.show()

from sklearn.decomposition import PCA
pca = PCA(n_components=df_ilr.shape[1])
pca.fit(df_ilr)
pca_df = pd.DataFrame(pca.transform(df_ilr)[:,[0,1,2,3]], index=df.index, columns=["PC1", "PC2", "PC3", "PC4"])
pca_df["sample_id"] = pca_df.index.get_level_values(0)
pca_df.set_index("sample_id", inplace=True)
pca_df = pca_df.join(sample_meta.set_index("sample_id"),  on="sample_id", how="left", lsuffix="_pca", rsuffix="_meta")
pca_df = pca_df.reset_index()
pca_df.Batch = [str(b) for b in pca_df.Batch]

fig, ax = plt.subplots(3, 2, figsize=(14, 12))
ax = ax.flatten()
for axis, label in zip(ax, ["Condition", "lesion_type", "Batch", "Sex", "Age", "lesion_type"]):
    sns.scatterplot(data=pca_df, x="PC1", y="PC2", hue=label, ax=axis)
    axis.set_title(f"PCA on Cluster Proportions, colored by {label}")
    axis.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    axis.set_xlabel(f"PC1: {np.round(pca.explained_variance_ratio_[0], 2)}")
    axis.set_ylabel(f"PC2: {np.round(pca.explained_variance_ratio_[1], 2)}")
    fig.tight_layout()
    axis.set_aspect('equal', 'box')
# plot sample_id labels
for string, x, y in zip(pca_df.sample_id, pca_df.PC1, pca_df.PC2):
    ax[5].text(x, y, string)
plt.show()


In [ ]:
for res in resolutions:
    df = adata.obs.groupby(["sample_id", f"leiden_{res}"]).size().unstack()
    df = df.div(df.sum(axis=1), axis=0)
    fig, ax = plt.subplots(figsize=(12, 8))
    sns.heatmap(df, cmap="viridis", annot=True, fmt=".2f", ax=ax)
    ax.set_xlabel("Cluster")
    ax.set_ylabel("Sample ID")
    fig.tight_layout()
    fig.suptitle(f"Cluster fractions per sample (resolution {res})")
    plt.show()
    #fig.savefig(plot_dir / f"cluster_fractions_per_sample_heatmap_res_{res}.pdf")

In [ ]:
for res in resolutions:
    df = adata.obs.groupby(["lesion_type", f"leiden_{res}"]).size().unstack()
    df = df.div(df.sum(axis=1), axis=0)
    fig, ax = plt.subplots(figsize=(12, 4))
    sns.heatmap(df, cmap="viridis", annot=True, fmt=".2f", ax=ax)
    ax.set_xlabel("Cluster")
    ax.set_ylabel("Lesion Type")
    fig.tight_layout()
    fig.suptitle(f"Cluster fractions per lesion type (resolution {res})")
    plt.show()
    #fig.savefig(plot_dir / f"cluster_fractions_per_sample_heatmap_res_{res}.pdf")

In [ ]:
for res in resolutions:
    df = adata.obs.groupby(["Condition", f"leiden_{res}"]).size().unstack()
    df = df.div(df.sum(axis=1), axis=0)
    fig, ax = plt.subplots(figsize=(12, 2))
    sns.heatmap(df, cmap="viridis", annot=True, fmt=".2f", ax=ax)
    ax.set_xlabel("Cluster")
    ax.set_ylabel("Sample ID")
    fig.tight_layout()
    fig.suptitle(f"Cluster fractions per condition (resolution {res})")
    plt.show()
    #fig.savefig(plot_dir / f"cluster_fractions_per_sample_heatmap_res_{res}.pdf")

In [ ]:
adata.obs

In [ ]:
obsm_key = obsm_to_use[0]
df = adata.obsm[obsm_key].copy()
meta_keys = ["sample_id", "Condition", "lesion_type"] + ["leiden_" + str(res) for res in resolutions]
df = df.join(adata.obs[meta_keys], on="cell")
df.reset_index(inplace=True)
df = df.melt(id_vars=["cell"]+meta_keys, var_name="feature", value_name="value")
for res in resolutions:
    fig, ax = plt.subplots(figsize=(8, 6))
    sns.heatmap(data=df.groupby(["feature", f"leiden_{res}"]).mean().reset_index().
                pivot(index="feature", columns=f"leiden_{res}", values="value"), 
                cmap="viridis")
    plt.show()

In [ ]:
obsm_key = obsm_to_use[1]
df = adata.obsm[obsm_key].copy()
meta_keys = ["sample_id", "Condition", "lesion_type"] + ["leiden_" + str(res) for res in resolutions]
df = df.join(adata.obs[meta_keys], on="cell")
df.reset_index(inplace=True)
df = df.melt(id_vars=["cell"]+meta_keys, var_name="feature", value_name="value")
for res in resolutions:
    df_tmp = df.groupby(["feature", f"leiden_{res}"]).mean().reset_index()
    df_tmp["min_max_value"] = df_tmp.groupby(["feature"]).value.apply(lambda x: (x - x.min()) / (x.max() - x.min()))
    fig, ax = plt.subplots(1, 2, figsize=(24, 12))
    sns.heatmap(data=df_tmp.pivot(index="feature", columns=f"leiden_{res}", values="value"), 
                cmap="viridis", ax=ax[0])
    sns.heatmap(data=df_tmp.pivot(index="feature", columns=f"leiden_{res}", values="min_max_value"), 
            cmap="viridis", ax=ax[1])
    ax[0].set_title("Raw values")
    ax[1].set_title("Min-max scaled values per feature")
    plt.show()

In [ ]:
obsm_key = obsm_to_use[0]
df = adata.obsm[obsm_key].copy()
df["leiden"] = adata.obs[["leiden_0.5"]]
df["condition"] = adata.obs.Condition.values
df.reset_index(inplace=True)
df = df.melt(id_vars=["cell", "leiden", "condition"], var_name="feature", value_name="value")

# compute mean feature per leiden and plot
fig, ax = plt.subplots(figsize=(6, 6))
sns.barplot(data=df, x="value", y="feature", hue="leiden", ax=ax)
ax.set_ylabel("Feature")
ax.set_xlabel("Mean feature value")
fig.tight_layout()

In [ ]:
obsm_key = obsm_to_use[1]
df = adata.obsm[obsm_key].copy()
df["leiden"] = adata.obs.leiden.values
df["condition"] = adata.obs.condition.values
df.reset_index(inplace=True)
df = df.melt(id_vars=["cell", "leiden", "condition"], var_name="feature", value_name="value")

# compute mean feature per leiden and plot
fig, ax = plt.subplots(figsize=(6, 12))
sns.barplot(data=df, x="value", y="feature", hue="leiden", ax=ax)
ax.set_ylabel("Feature")
ax.set_xlabel("Mean feature value")
fig.tight_layout()

# Feature Loadings per Factor

In [ ]:
model.get_factors().shape

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
sns.heatmap(model.get_weights(df=True).loc[obsm_features["abunds_all"], :], ax=ax, cmap="coolwarm", center=0)
plt.show()

# same heatmap as above but standardize each column (subtract mean and divide by std)
#fig, ax = plt.subplots(1, 1, figsize=(6, 6))
#df = model.get_weights(df=True).loc[obsm_features["abunds_all"], :]
#df = df.apply(lambda x: (x - x.mean()) / x.std(), axis=0)
#sns.heatmap(df, ax=ax, cmap="coolwarm", center=0)
#plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 10))
sns.heatmap(model.get_weights(df=True).loc[obsm_features["hallmark_estimates"], :], ax=ax, cmap="coolwarm", center=0)
plt.show()

# same heatmap as above but standardize each column (subtract mean and divide by std)
#xfig, ax = plt.subplots(1, 1, figsize=(6, 10))
#xdf = model.get_weights(df=True).loc[obsm_features["hallmark_estimates"], :]
#xdf = df.apply(lambda x: (x - x.mean()) / x.std(), axis=0)
#xsns.heatmap(df, ax=ax, cmap="coolwarm", center=0)
#xplt.show()

# Distribution of Factors per Sample or Condition

In [ ]:
df = model.get_factors(df=True)
df["sample_id"] = meta_df.loc[model.get_cells().cell, "sample_id"].values
df["condition"] = meta_df.loc[model.get_cells().cell, "condition"].values
df["lesion_type"] = meta_df.loc[model.get_cells().cell, "lesion_type"].values
df = df.reset_index().melt(id_vars=["index", "sample_id", "condition", "lesion_type"], var_name="Factor", value_name="value")

In [ ]:
g = sns.FacetGrid(data=df, col="Factor", hue="condition", col_wrap=3, sharex=False, sharey=False, legend_out=True)
g.map_dataframe(sns.histplot, x="value", stat="density", common_norm=False, bins=100, alpha=0.5)
g.tight_layout()
g.add_legend()
plt.show()

In [ ]:
#g = sns.FacetGrid(data=df, row="condition", col="Factor", legend_out=True)
#g.map_dataframe(sns.histplot, x="value", stat="density", common_norm=False, bins=100)
#g.tight_layout()
#g.add_legend()
#plt.show()
#
#g = sns.FacetGrid(data=df, row="sample_id", col="Factor", legend_out=True)
#g.map_dataframe(sns.histplot, x="value", stat="density", common_norm=False, bins=100)
#g.tight_layout()
#g.add_legend()
#plt.show()

# Correlation of Factors

In [ ]:
# the factors are mutally independent?
mfx.plot_factors_correlation(model)

# R2 Values

In [ ]:
df_r2 = model.get_r2()

In [ ]:
mfx.plot.plot_r2(model, y="Group", x="Factor")
#mfx.plot_r2_barplot(model, factors=list(range(0, 6)), x="Group", groupby="Factor", palette="winter")

In [ ]:
df_r2

In [ ]:
# group df_r2 by Group and Factor and take the geometric mean of the R2 values
stat = df_r2.drop("View", axis=1).groupby(["Group", "Factor"]).apply(lambda x: np.prod(x)**(1/len(x))).groupby("Group").sum().R2.mean()
stat

# Random

In [ ]:
plt.figure(figsize=(4, 10))
ax = mfx.plot_weights(model, n_features=2, y_repel_coef=0.04, x_rank_offset=-150, views=["abunds_all"])
plt.show()

In [ ]:
plt.figure(figsize=(4, 10))
ax = mfx.plot_weights(model, n_features=2, y_repel_coef=0.04, x_rank_offset=-150, views=["hallmark_estimates"])
plt.show()

In [ ]:
mfx.plot_weights_heatmap(model, n_features=5, view=0,
                         factors=range(0, 9), 
                         xticklabels_size=6, w_abs=True, 
                         cmap="viridis", cluster_factors=False)

In [ ]:
mfx.plot_weights_heatmap(model, n_features=5, view=1,
                         factors=range(0, 9), 
                         xticklabels_size=6, w_abs=True, 
                         cmap="viridis", cluster_factors=False)

In [ ]:
mfx.plot_weights_dotplot(model, n_features=3, 
                         w_abs=True, 
                         factors=list(range(0, 9)), yticklabels_size=5)